In [1]:
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
import keras
from nltk.corpus import stopwords
import collections
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix

ImportError: Unable to import required dependencies:
pytz: No module named 'pytz'

In [4]:
import tensorflow-gpu as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

SyntaxError: invalid syntax (1096613164.py, line 1)

In [ ]:
data = pd.read_csv("spam_ham_dataset.csv", encoding='latin')
data = data.drop(columns=['Unnamed: 0'])
data = data.dropna(axis=0)

In [ ]:
def word_count_plot(data):
    # finding words along with count
     word_counter = collections.Counter([word for sentence in data for word in sentence.split()])
     most_count = word_counter.most_common(30) # 30 most common words
     # sorted data frame
     most_count = pd.DataFrame(most_count, columns=["Word", "Count"]).sort_values(by="Count")
     most_count.plot.barh(x = "Word", y = "Count", color="green", figsize=(10, 15))
        
word_count_plot(data['text'])

In [ ]:
# does preprocessing
lem = WordNetLemmatizer()
def preprocess(data):
    mail = data.lower()
    mail = re.sub("[^a-z ]", "", mail)
    mail = mail.split()
    mail = [lem.lemmatize(word) for word in mail if not word in set(stopwords.words('english'))]
    mail = " ".join(mail)
    return mail

x = data['text'].apply(preprocess)
word_count_plot(x)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(data['label'])
y.shape

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)
text2seq = tokenizer.texts_to_sequences(x)

maxseq = max([len(i) for i in text2seq])
padded = pad_sequences(text2seq, maxlen=maxseq, padding='pre')
padded.shape

In [ ]:
# create training and testing sets
x_train, x_test, y_train, y_test = train_test_split(padded, y, random_state=42, test_size=0.2)

In [ ]:
# create model
TOT_SIZE = len(tokenizer.word_index) + 1

lstm = Sequential()
lstm.add(Embedding(TOT_SIZE, 32, input_length=maxseq))
lstm.add(LSTM(100, dropout=0.4))
'''lstm.add(Dropout(0.4))
lstm.add(Dense(20, activation='relu'))
lstm.add(Dropout(0.3))'''
lstm.add(Dense(1, activation='sigmoid'))

lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm.summary()

In [ ]:
# train
lstm.fit(x_train, y_train, epochs=2, validation_split=0.2, batch_size=16)

In [ ]:
# evaluate
y_pred = lstm.predict(x_test)
y_pred = (y_pred > 0.5)
print("Test Score:{:.2f}%".format(accuracy_score(y_test, y_pred)*100))
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)